# Part C: High-Throughput Segmentation for RT-DC

## 🔬 Overview
This notebook segments individual high-speed microscopy frames filtered during Part A. By using **Omnipose**, we achieve high-fidelity boundary detection of deformed cells, which is critical for accurate mechanical phenotyping.

### Pipeline Stages:
1. **Input:** Filtered `.tif` image sequences from the microfluidic detection zone.
2. **Processing:** Deep-learning segmentation using the `bact_phase_omni` model.
3. **Output:** Generation of 16-bit labeled masks for feature extraction (Area, Deformability, etc.).

In [ ]:
import os
import numpy as np
from skimage.io import imread, imsave
from omnipose import core
from cellpose import models

# =============================================================================
# 1. DEFINE GENERIC PATHS
# =============================================================================
base_dir = os.path.dirname(os.getcwd())
input_path = os.path.join(base_dir, "data", "raw", "all_cells")
mask_path = os.path.join(base_dir, "data", "processed", "masks")

# Auto-create output directory
os.makedirs(mask_path, exist_ok=True)

# Load image sequence
if not os.path.exists(input_path):
    print(f"Error: Input folder not found at {input_path}")
else:
    files = sorted([os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith('.tif')])
    print(f"Detected {len(files)} frames for high-throughput processing.")

# =============================================================================
# 2. OMNIPOSE EVALUATION
# =============================================================================
# Initialize model - 'bact_phase_omni' is optimized for phase-contrast RT-DC channels
model = models.CellposeModel(gpu=False, model_type='bact_phase_omni')

for f_path in files:
    img = imread(f_path)
    fname = os.path.basename(f_path)
    
    # Segment the frame
    # omni=True and resample=True ensure high-fidelity boundaries for deformed shapes
    masks, flows, styles = model.eval(img, 
                                      channels=[0,0], 
                                      rescale=None, 
                                      mask_threshold=0.0, 
                                      flow_threshold=0.4, 
                                      omni=True)
    
    # Save the labeled mask (16-bit to support high cell counts per frame if needed)
    save_name = os.path.join(mask_path, fname.replace('.tif', '_mask.png'))
    imsave(save_name, masks.astype(np.uint16), check_contrast=False)

print("--- Segmentation Complete. Ready for Feature Extraction (Code 2) ---")